In [1]:
#! /usr/bin/python3
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import *

In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import random, sys


In [4]:
%matplotlib inline
%matplotlib notebook
plt.ioff()

In [5]:
env = gym_super_mario_bros.make('SuperMarioBros-1-3-v0')
env = JoypadSpace(env, RIGHT_ONLY)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [6]:
#Animation
save_animation = 1 # Saves animation to plot
frames_skipped = 1 # How many frames to skip when creating animation

#Generation
generation_amount = 1000

#Population
population_amount = 10 # Amount of agents per generation

#Agent
moves_amount = 5000 # Max amount of moves agent can perform
moves_to_check = 30 # How many moves without increase in fitness until termination
mutation_rate = 1.001 # Chance of any move changing to a random move
moves_mutable = 0.2 # How many percent of moves are mutable (starting from end)
                    # Regardless of percent, a minimum of (moves_to_check*2) moves
                    # are mutable.

In [7]:
t = 0
def print_info(info, reward):
    global t 
    t += 1
    if not t % 100:
        print(info, reward)
def handle_frame(step, images):
    if not step % frames_skipped:
        image = plt.imshow(env.render(mode='rgb_array'))
        images.append([image])
fig = plt.figure()
def display_animation(images):
    anim = animation.ArtistAnimation(fig, images,
                                     interval=15*frames_skipped, blit=True)
    rc('animation', html='jshtml')
    return(anim)
class Agent:
    def __init__(self, fittest):
        self.fitness = 0
        self.images = []
        if fittest:
            self.moves = fittest.moves.copy()
            self.moves_used = fittest.moves_used
        else:
            self.moves = []
            self.moves_used = 0
def check_fitness(player, fitness, past_fitness):
    if player.fitness < fitness:
        player.fitness = fitness
    if len(past_fitness) < moves_to_check:
        past_fitness.append(fitness)
    else:
        past_fitness.pop(0)
        past_fitness.append(fitness)
        for i in range(moves_to_check):
            if past_fitness[i] > past_fitness[0]:
                break
            #Kills player if no progress in fitness for moves_to_check moves
            if i == moves_to_check - 1:
                return True
def mutate_moves(player):
    start_index = int(player.moves_used * (1-moves_mutable))
    if player.moves_used - start_index < moves_to_check * 2:
        start_index = player.moves_used - moves_to_check * 2
    for i in range(start_index, moves_amount):
        num = random.random()
        if num < mutation_rate:
            player.moves[i] = env.action_space.sample()

def agent_get_moves(player, population):
    if not player.moves:
        for i in range(moves_amount):
            player.moves.append(env.action_space.sample())
    else:
        if population:
            mutate_moves(player)
def print_mutation_info(fittest, population):
    global mutation_rate
    current_fittest = get_fittest(population)
    mutable_amount = current_fittest.moves_used * moves_mutable # new line
    if fittest and fittest.fitness == current_fittest.fitness:
        print("Increasing mutation chance by 0.005")
        mutation_rate += 0.005
        print("Mutation chance is now {}%".format(mutation_rate*100))
    else:
        print("Fitness increased succesfully, restore mutation chance to 0.01")
        mutation_rate = 0.01
        mutable_amount = current_fittest.moves_used * moves_mutable

    if mutable_amount < moves_to_check * 2: 
        mutable_amount = moves_to_check * 2  
        

    print("Mutating last {}% of moves, equal to {} moves".format(moves_mutable*100,
                                                                 mutable_amount))
def display_recording(player):
    print("Displaying player with fitness score {}.".format(
                                    player.fitness))
    display_animation(player)
def get_fittest(population):
    player_num = 0
    for i in range(1, len(population)):
        if population[i].fitness > population[player_num].fitness:
            player_num = i
    return population[player_num]

def record_player(player):
    images = []
    done = False
    observation = env.reset()
    past_fitness = []
    for move in range(len(player.moves)):
        if done:
            break
        state, reward, done,info = env.step(player.moves[move])
        # state, reward, done,info = env.step(player.moves[move])
        if done:
            break
        done = check_fitness(player, info['x_pos'], past_fitness)
        handle_frame(move, images)
    return images
        
def play_generation(fittest, population):
    for i in range(population_amount):
        player = Agent(fittest)
        observation = env.reset()
        done = False
        past_fitness = []
        agent_get_moves(player, population)
        for move in range(len(player.moves)):
            if done:
                break
            state, reward, done,info = env.step(player.moves[move])
            # state, reward, done,info = env.step(player.moves[move])
            if done:
                break
            done = check_fitness(player, info['x_pos'], past_fitness)
            env.render()
        player.moves_used = move
        population.append(player)
        print("Player {} achieved a fitness of {} in {} moves".format(i,
                                                               player.fitness, move))
        if info['flag_get']:
            return 1
    print_mutation_info(fittest, population)
    return 0

In [8]:
def custom_starting_agent(filename, fitness):
    fittest = Agent(None)
    fittest.fitness = fitness
    fittest.moves = open(filename, "r").read()
    fittest.moves = fittest.moves.strip("[]")
    fittest.moves = [int(s) for s in fittest.moves.split(',')]
    fittest.moves_used = 1120
    return (fittest)


In [9]:
fittest = None
#fittest = custom_starting_agent("2227-fitness", 900)
for generation in range(generation_amount):
    population = []
    if play_generation(fittest, population):
        break
    fittest = get_fittest(population)
    print("GENERATION {} HIGHEST FITNESS ACHIEVED: {}".format(generation,
                                                             fittest.fitness))

2023-10-13 11:19:48.413 Python[4617:101378] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/xy/pc1nynp1589_1lzrzwtd13_h0000gn/T/org.python.python.savedState
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Player 0 achieved a fitness of 281 in 133 moves
Player 1 achieved a fitness of 278 in 144 moves
Player 2 achieved a fitness of 408 in 163 moves
Player 3 achieved a fitness of 422 in 166 moves


2023-10-13 11:19:55.269 Python[4617:101378] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Player 4 achieved a fitness of 313 in 156 moves
Player 5 achieved a fitness of 283 in 143 moves
Player 6 achieved a fitness of 388 in 174 moves
Player 7 achieved a fitness of 406 in 173 moves
Player 8 achieved a fitness of 408 in 178 moves
Player 9 achieved a fitness of 311 in 136 moves
Fitness increased succesfully, restore mutation chance to 0.01
Mutating last 20.0% of moves, equal to 60 moves
GENERATION 0 HIGHEST FITNESS ACHIEVED: 422
Player 0 achieved a fitness of 422 in 166 moves
Player 1 achieved a fitness of 422 in 166 moves
Player 2 achieved a fitness of 422 in 166 moves
Player 3 achieved a fitness of 422 in 166 moves
Player 4 achieved a fitness of 422 in 166 moves
Player 5 achieved a fitness of 422 in 166 moves
Player 6 achieved a fitness of 422 in 166 moves
Player 7 achieved a fitness of 422 in 166 moves
Player 8 achieved a fitness of 422 in 166 moves
Player 9 achieved a fitness of 422 in 166 moves
Increasing mutation chance by 0.005
Mutation chance is now 1.5%
Mutating last 

In [ ]:
winner = get_fittest(population)
print("Player from generation {} won! Achieving a fitness of {} in {} moves!"
          .format(generation, winner.fitness, winner.moves_used))
print("Compiling animation...")
ani = display_animation(record_player(winner))
ani

In [ ]:
Writer = animation.FFMpegFileWriter
writer = Writer(fps=20, metadata=dict(artist='Me'), bitrate=-1)
ani.save("mario_flag_get-{}.mp4".format(random.random()), writer = writer)

In [ ]:
env.close()
plt.close('all')